In [1]:
import cv2
import numpy as np
from keras.datasets import mnist

/home/ekele/.conda/envs/gpudeeplearning/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# Training Parameters
batch_size = 128
epochs = 1

#img_rows = x_train.shape[0]
#img_cols = x_train.shape[0]
x_train.shape

(60000, 28, 28)

In [4]:
x_train[1].shape[0]

28

In [5]:
x_train.shape[0]

60000

In [6]:
x_train[1].shape[0]

28

In [7]:
y_train.shape

(60000,)

In [8]:
img_rows = x_train[0].shape[0]
img_cols = x_train[1].shape[0]

#reshaping to 4 dimensions for keras
x_train = x_train.reshape(x_train.shape[0],img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

# shape of a single image
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize our data by changing the range from (0 to 255) to (0 to 1)
x_train /= 255
x_test /= 255

# Converting to categorical data
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# Let's count the number columns in our hot encoded matrix 
print ("Number of Classes: " + str(y_test.shape[1]))

Number of Classes: 10


In [9]:
#y_test.shape[1]
x_train.shape[0]

60000

In [10]:
num_classes = y_test.shape[1]
num_pixels = x_train.shape[1] * x_train.shape[2]

In [11]:
import keras
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = keras.optimizers.Adam(),
              metrics = ['accuracy'])

print (model.summary())

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 36864)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               4718720   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total para

#### Serializing

import pickle

pickle_out = open("MNIST_history.pickle","wb")
pickle.dump(history.history, pickle_out)
pickle_out.close()

#### Loading saved history

In [13]:
import pickle

pickle_in = open("MNIST_history.pickle", "rb")
saved_history = pickle.load(pickle_in)
print(saved_history)

{'val_loss': [0.045635181629657746], 'acc': [0.9402333333651225], 'loss': [0.1997614995956421], 'val_acc': [0.9854]}


#### Displaying confusion matrix and Classification Report

In [14]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

y_pred = model.predict_classes(x_test)
print(classification_report(np.argmax(y_test,axis=1), y_pred))
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

             precision    recall  f1-score   support

          0       0.99      0.99      0.99       980
          1       0.99      0.99      0.99      1135
          2       0.98      0.98      0.98      1032
          3       0.98      0.99      0.98      1010
          4       0.99      0.98      0.99       982
          5       0.97      0.99      0.98       892
          6       0.99      0.98      0.99       958
          7       0.99      0.98      0.98      1028
          8       0.99      0.97      0.98       974
          9       0.98      0.97      0.98      1009

avg / total       0.98      0.98      0.98     10000

[[ 968    0    5    0    0    1    3    1    2    0]
 [   0 1127    2    1    0    2    1    1    1    0]
 [   0    4 1015    6    0    0    0    6    1    0]
 [   0    0    1 1003    0    4    0    1    1    0]
 [   0    0    1    0  966    0    5    0    2    8]
 [   0    0    0    5    0  886    1    0    0    0]
 [   7    2    0    1    2    6  940    0  

##### Displaying misclassified data

In [16]:
import cv2
import numpy as np

(x_train, y_train), (x_test, y_test)  = mnist.load_data()
result = np.absolute(y_test - y_pred)
result

array([0, 0, 0, ..., 0, 0, 0])

In [18]:
result_indices = np.nonzero(result > 0) # meaning misclassified
result_indices
print("Indices of misclassified data are: \n\n" + str(result_indices))

Indices of misclassified data are: 

(array([ 115,  247,  259,  321,  359,  445,  582,  619,  684,  691,  716,
        717,  740,  846,  883,  924,  947,  965, 1014, 1039, 1112, 1182,
       1202, 1226, 1232, 1247, 1260, 1319, 1326, 1364, 1393, 1500, 1522,
       1527, 1530, 1549, 1553, 1621, 1681, 1709, 1717, 1722, 1754, 1878,
       1901, 1903, 2016, 2035, 2043, 2053, 2098, 2118, 2130, 2135, 2182,
       2189, 2293, 2329, 2387, 2406, 2422, 2447, 2454, 2597, 2607, 2654,
       2896, 2927, 2939, 2953, 2995, 3012, 3030, 3062, 3073, 3289, 3330,
       3503, 3520, 3559, 3597, 3674, 3718, 3727, 3751, 3757, 3767, 3780,
       3808, 3811, 3850, 3853, 3906, 3941, 4007, 4065, 4075, 4176, 4199,
       4205, 4224, 4238, 4248, 4360, 4497, 4511, 4536, 4571, 4575, 4601,
       4639, 4671, 4740, 4761, 4807, 4874, 4956, 4978, 5140, 5331, 5634,
       5642, 5887, 5937, 5955, 6071, 6091, 6166, 6505, 6555, 6560, 6576,
       6597, 6603, 6625, 6651, 6755, 7121, 7216, 7434, 8059, 8069, 8094,
       8273, 

In [22]:
result_indices[0][1]

247

In [24]:
help(cv2.resize)

Help on built-in function resize:

resize(...)
    resize(src, dsize[, dst[, fx[, fy[, interpolation]]]]) -> dst
    .   @brief Resizes an image.
    .   
    .   The function resize resizes the image src down to or up to the specified size. Note that the
    .   initial dst type or size are not taken into account. Instead, the size and type are derived from
    .   the `src`,`dsize`,`fx`, and `fy`. If you want to resize src so that it fits the pre-created dst,
    .   you may call the function as follows:
    .   @code
    .   // explicitly specify dsize=dst.size(); fx and fy will be computed from that.
    .   resize(src, dst, dst.size(), 0, 0, interpolation);
    .   @endcode
    .   If you want to decimate the image by factor of 2 in each direction, you can call the function this
    .   way:
    .   @code
    .   // specify fx and fy and let the function compute the destination image size.
    .   resize(src, dst, Size(), 0.5, 0.5, interpolation);
    .   @endcode
    .   To shrin

#### Displaying the misclassifications

In [25]:
import cv2

def draw_test(name, pred, input_im, true_label):
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(input_im, 0, 0, 0, imageL.shape[0]*2, cv2.BORDER_CONSTANT, value=BLACK)
    expanded_image = cv2.cvtColor(expanded_image, cv2.COLOR_GRAY2BGR)
    cv2.putText(expanded_image, str(pred), (152, 70), cv2.FONT_HERSHEY_COMPLEX_SMALL, 4, (0,255,0), 2, cv2.LINE_AA)
    cv2.putText(expanded_image, str(true_label), (250, 70), cv2.FONT_HERSHEY_COMPLEX_SMALL, 4, (0,0,255), 2)
    cv2.imshow(name, expanded_image)
    
for i in range(0,10):
    
    input_im = x_test[result_indices[0][i]]
    imageL = cv2.resize(input_im, None, fx=4, fy=4, interpolation=cv2.INTER_CUBIC)
    input_im = input_im.reshape(1,28,28,1)
    
    res = str(model.predict_classes(input_im, 1, verbose = 0)[0])
    draw_test("Prediction", res, imageL, y_test[result_indices[0][i]])
    cv2.waitKey(0)
    
cv2.destroyAllWindows()